In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import pandas_gbq
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr


from time import sleep

import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials, 
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'     # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



In [ ]:
# kor_index_ohlcv
# kor_index_code_fundamental
# kor_index_list_df

In [4]:
glob.glob('./data_crawler/kor_index_ohlcv/*')

['./data_crawler/kor_index_ohlcv/kor_index_ohlcv.csv']

In [9]:
for file_nm in ['kor_index_ohlcv','kor_index_code_fundamental','kor_index_list_df']:
    print(file_nm)
    df = pd.read_csv(f'./data_crawler/{file_nm}/{file_nm}.csv')
    table_from_pandas = pa.Table.from_pandas(df,preserve_index = False)
    pq.write_table(table_from_pandas, f'./data_crawler/{file_nm}/{file_nm}.parquet')

    # Google Storage 적재
    source_file_name = f'./data_crawler/{file_nm}/{file_nm}.parquet'   # GCP에 업로드할 파일 절대경로
    destination_blob_name = f'./data_crawler/{file_nm}/{file_nm}.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    

kor_index_ohlcv


/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


kor_index_code_fundamental
kor_index_list_df


/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [5]:
glob.glob('./data_crawler/kor_index_code_fundamental/*')

['./data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.csv']

In [6]:
glob.glob('./data_crawler/kor_index_list_df/*')

['./data_crawler/kor_index_list_df/kor_index_list_df.csv']